In [3]:
%pwd

'/workspaces/End-End-Text-summeriser'

In [2]:
# change working directory to root directory
import os
os.chdir("../")

In [ ]:
# update the config/config.yaml file for data transformation 
# convert original data into 
# #input_id (tokenization, input_encoding), 
# attention_mask (tokenization, input_encoding), 
# labels (tokenization, target_encoding).

In [ ]:
# 3. define the entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path # root directory for data transformation
    data_path: Path # path to the original data used for data transformation
    tokenizer_name: Path # path to the tokenizer used for data transformation, model will be automatically downloaded.

In [ ]:
# 4. configuration manager in src config.

In [6]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    #constructor assigning values from constant file to the class variables


    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # reading yaml config file and params file and assigning them to the class variables

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create directories/file for artifacts
        # note that we are using . to access, as we have defined using configbox

        create_directories([self.config.artifacts_root])


    # user defined functions with parameters returned type are based on the decorator function defined in above cell
    # returns the 3 variables
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir]) # create directories for data transformation

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [ ]:
#5. components

In [10]:
import os
from textSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [19]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config # initialize the config object
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name) # initialize the tokenizer object


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    # function to load the data from disk, convert, and save the data to disk

    def convert(self):
        logger.info(f"Loading data from {self.config.data_path}")
        dataset_samsum = load_from_disk(self.config.data_path)
        logger.info(f"Loaded train:{len(dataset_samsum['train'])},test:{len(dataset_samsum['test'])},validation:{len(dataset_samsum['validation'])} examples")
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        logger.info(f"Transformed train:{len(dataset_samsum['train'])},test:{len(dataset_samsum['test'])},validation:{len(dataset_samsum['validation'])} examples")
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        logger.info(f"Saved train:{len(dataset_samsum['train'])},test:{len(dataset_samsum['test'])},validation:{len(dataset_samsum['validation'])} examples to {self.config.root_dir}/samsum_dataset")

In [ ]:
# 6. pipeline

In [20]:
try:
    config = ConfigurationManager() # initialize the configuration manager object
    data_transformation_config = config.get_data_transformation_config() # initialize the data transformation config object
    data_transformation = DataTransformation(config=data_transformation_config) # initialize the data transformation object
    data_transformation.convert() # run the data transformation
except Exception as e:
    raise e

[2024-03-19 01:39:01,415: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-19 01:39:01,417: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-19 01:39:01,418: INFO: common: created directory at: artifacts]
[2024-03-19 01:39:01,419: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-19 01:39:02,264: INFO: 2754396705: Loading data from artifacts/data_ingestion/samsum_dataset]
[2024-03-19 01:39:02,271: INFO: 2754396705: Loaded train:14732,test:819,validation:818 examples]


Map: 100%|██████████| 818/818 [00:00<00:00, 2414.43 examples/s]

[2024-03-19 01:39:08,414: INFO: 2754396705: Transformed train:14732,test:819,validation:818 examples]



Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 97226.84 examples/s] 

[2024-03-19 01:39:08,526: INFO: 2754396705: Saved train:14732,test:819,validation:818 examples to artifacts/data_transformation/samsum_dataset]


In [ ]:
# pipeline created successfully for data transformation, can be verified under artifacts/data_transformation/samsum_dataset
# Update components for data validation in the src folder.